### 드레스 피팅 대화 데이터셋을 바탕으로 키워드 추출
- 키워드 추출이 의미가 있을까?
- 그냥 바로 이미지 제작을 위한 프롬프트 생성 해버리는게 나을 것 같아서 그렇게 진행

In [32]:
dialogue = """
<드레스 착용 전>
실장 : 신부님 피부 톤이 살짝 있으셔서 완전 화이트보다는 살짝 아이보리 톤이 들어가고 광택감이 있는 미카도 실크가 어울리실 것 같아요. 혹시 드레스 선택하실 때 생각했던 디자인 있으실까요?
신부 : 제가 팔뚝 살이 좀 있어서 팔을 좀 가리는 디자인이면 좋겠어요.
실장 : 슬리브 있는 드레스는 오히려 팔을 드러낼 수 있어서 오프숄더 디자인이 오히려 팔이 얇아보이실거에요. 오프숄더 디자인으로 한번 준비해드릴게요.
<드레스 착용 후>
실장 : 너무 잘 어울리세요. 베일도 투 베일로 해서 베일로 시선 집중을 시켜서 팔이 더 얇아보이는 효과가 있어요. 신부님이 키가 있으셔서 살짝 슬림한 A라인으로 떨어지는 디자인이 날씬하게 보이는 효과가 있어요. 혹시 티아라는 하실건가요?
신부 : 아뇨, 티아라는 괜찮아요.
실장 : 맞아요. 이 드레스가 작은 비즈가 촘촘히 박혀있어서, 아래쪽이 화려하다보니 티아라까지 하면 좀 투머치일 수 있어요. 식장은 어디쪽으로 잡으셨어요?
신부 : 빌라드지디 청담으로 잡았어요.
실장 : 빌라드지디가 밝은 홀이라 레이스 없이 깔끔한 이런 디자인이 잘 어울려요. 식장이랑도 잘 어울리겠네요.
"""
system_prompt = """
기본정보와 상세 디자인 정보를 바탕으로 대화에 나오는 드레스에 대해 최대한 상세하게 설명해주세요.

----
### FORMAT :
{
  '기본정보' : {
    '스타일' : '클래식',
    '원단' : '미카도 실크',
    '컬러' : '아이보리'
  },
  '상세 디자인': {
    '스타일' : '오프숄더',
    '스트랩' : '없음',
    '디테일' : '비즈 장식'
  }
  '설명' : '${드레스묘사}'
}
-----
### 반드시 JSON 형식으로 답변하세요.
  
### Description :
"""


# img_maker_prompt = """
# You are an expert Prompt Writer for image generation. 
# Your goal is to create a prompt based on the given dialogue to guide the image generator in creating a sketch:

# --------------------

# Dialogue: {dialogue}

# --------------------

# Guidelines for crafting effective prompts:

# - Use simple and clear language that the generator can easily understand.
# - Keep your prompts concise, avoiding overly detailed descriptions that might confuse the system.
# - Incorporate descriptive words that capture the subject and layout of the image.
# - Avoid contradictory terms that could confuse the AI in determining the style or content.

# ---------

# ### EXAMPLE:
# A beautiful, flowing ball gown with a classic silhouette, square neckline, and cinched waist. Emphasize the elegant drape of the fabric and the full skirt. Sketch style, black and white line art.

# -----

# Now, create a prompt.

# ### REMEMBER to add "Sketch style, black and white line art" at the end of the prompt.
# ### REMEMBER: Answer in ENGLISH
# ### PROMPT:
# """
img_maker_prompt = """
You are an expert Prompt Writer for image generation. 
Your goal is to create a prompt based on the given dialogue to guide the image generator in creating a sketch:

--------------------

Dialogue: {dialogue}

--------------------

Guidelines for crafting effective prompts:

- Focus on the front view of the clothing only. Ensure the prompt emphasizes the design, details, and features of the clothing from the front.
- Make sure that no person or human figure is included in the sketch, only the clothing itself.
- Use simple and clear language that the generator can easily understand.
- Keep your prompts concise, avoiding overly detailed descriptions that might confuse the system.
- Incorporate descriptive words that capture the subject and layout of the clothing.
- Avoid contradictory terms that could confuse the AI in determining the style or content.

---------

### EXAMPLE:
A front view of a beautiful, flowing ball gown with a classic silhouette, square neckline, and cinched waist. Emphasize the elegant drape of the fabric and the full skirt. No human figure included. Sketch style, black and white line art.

-----

Now, create a prompt.

### REMEMBER to add "Sketch style, black and white line art" at the end of the prompt.
### REMEMBER: Answer in ENGLISH
### PROMPT:
"""

In [5]:
from openai import OpenAI # openai==1.2.0
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv('UPSTAGE_API_KEY'),
    base_url="https://api.upstage.ai/v1/solar"
)
 
stream = client.chat.completions.create(
    model="solar-1-mini-chat",
    messages=[
      {
        "role": "system",
        "content": system_prompt
      },
      {
        "role": "user",
        "content": dialogue
      }
    ],
    stream=True,
)

result = ''
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        text = chunk.choices[0].delta.content
        print(text, end="")
        result += text

# print(result)
# Use with stream=False
# print(stream.choices[0].message.content)

{
  '기본정보' : {
    '스타일' : 'A라인',
    '원단' : '미카도 실크',
    '컬러' : '아이보리'
  },
  '상세 디자인': {
    '스타일' : '오프숄더',
    '스트랩' : '없음',
    '디테일' : '비즈 장식'
  }
  '설명' : '아이보리 컬러의 미카도 실크 소재의 A라인 드레스로, 오프숄더 디자인과 비즈 장식이 특징입니다. 슬림한 실루엣으로 키가 큰 신부에게 잘 어울리며, 티아라 없이도 화려한 느낌을 줍니다. 빌라드지디 청담과 같은 밝은 홀에서 잘 어울리는 드레스입니다.'
}

### 위에서 만들어진 프롬프트를 기반으로 이미지를 생성하라고 시킴
- 사용한 모델 : flux
- https://fal.ai/models/fal-ai/flux/dev/examples

In [17]:
os.environ['FAL_KEY'] = os.getenv('FAL_KEY')
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

In [18]:
from langchain_upstage import ChatUpstage
from langchain import LLMChain
from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI

In [33]:
def make_prompt(dialogue):
    template = img_maker_prompt
    prompt_template = PromptTemplate(input_variables=["dialogue"], template=template)
    chain = LLMChain(prompt=prompt_template, llm=ChatUpstage(api_key=UPSTAGE_API_KEY))
    response = chain.run(dialogue=dialogue)
    return response

In [34]:
img_prompt = make_prompt(dialogue)
img_prompt

"A front view of a bridal dress in a slightly ivory tone with a glossy mikado silk texture. The dress features an off-shoulder design with a slightly slimmer A-line shape that falls gracefully to the floor. The dress has a fitted bodice with a cinched waist, accentuating the bride's figure. The skirt is full and flared, with a small train at the back. The dress has a simple, elegant design, with a subtle pattern of small, 촘촘히 박혀있는 비즈. The dress is paired with a veil that is held up by a headpiece, adding a touch of elegance to the overall look. No human figure included. Sketch style, black and white line art."

In [35]:
import fal_client

handler = fal_client.submit(
    "fal-ai/flux/dev",
    arguments={
        "prompt": img_maker_prompt,
        "image_size": "landscape_4_3",
        "num_inference_steps": 28,
        "guidance_scale": 3.5,
        "num_images": 1,
        "enable_safety_checker": True
    },
)

result = handler.get()
print(result)

{'images': [{'url': 'https://fal.media/files/lion/VxlsjwZ0g1iw8Q-YKFN-C.png', 'width': 1024, 'height': 768, 'content_type': 'image/jpeg'}], 'timings': {'inference': 2.200373040002887}, 'seed': 4077771586, 'has_nsfw_concepts': [False], 'prompt': '\nYou are an expert Prompt Writer for image generation. \nYour goal is to create a prompt based on the given dialogue to guide the image generator in creating a sketch:\n\n--------------------\n\nDialogue: {dialogue}\n\n--------------------\n\nGuidelines for crafting effective prompts:\n\n- Focus on the front view of the clothing only. Ensure the prompt emphasizes the design, details, and features of the clothing from the front.\n- Make sure that no person or human figure is included in the sketch, only the clothing itself.\n- Use simple and clear language that the generator can easily understand.\n- Keep your prompts concise, avoiding overly detailed descriptions that might confuse the system.\n- Incorporate descriptive words that capture the 

### 결과

In [36]:
img_url = result['images'][0]['url']

In [37]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= img_url)

### 정리
- 스케치 결과는 어찌어찌 나온다

문제점:
- 매번 스케치의 형태가 다르게 나온다
- 틀린 형태가 나오는 것 같기도 하다 (판단이 어렵긴 하다)

TO-DO:
- 사람 안나오도록??